In [1]:
from functools import reduce
def to_bits(*l):
    return reduce(lambda r,i: r |(1<<i), l, 0)

def winning_patterns():
    v1 = to_bits(0,1,2)
    h1 = to_bits(0,3,6)
    return [v1, v1<<3, v1<<6, h1, h1<<1, h1<<2, to_bits(0,4,8), to_bits(2,4,6)]
WINNING_PATTERNS = winning_patterns()

In [2]:
import numpy as np
import theano
import theano.tensor as T

import lasagne
floatX = theano.config.floatX
floatX

Using gpu device 0: GeForce GTX 965M (CNMeM is disabled)


'float32'

In [3]:
all_input_data = np.zeros((2**9, 1, 3, 3), dtype=floatX)
all_target_data = np.zeros(2**9, dtype=floatX)
for board in range(2**9):
    b = 1
    for i in range(3):
        for j in range(3):
            all_input_data[board, 0, i, j] = bool(board & b)
            b <<= 1
    all_target_data[board] = any(p&board == p for p in WINNING_PATTERNS)


In [4]:
#theano.config.exception_verbosity = "high"
#theano.config.optimizer= 'None'

In [5]:
input_var = T.tensor4('inputs')
target_var = T.vector('targets')
l_in = lasagne.layers.InputLayer(shape=(None, 1, 3, 3), input_var=input_var)
#l_in_drop = lasagne.layers.DropoutLayer(l_in, p=0.2)
l_hidden = lasagne.layers.DenseLayer(l_in, num_units=64, nonlinearity=lasagne.nonlinearities.tanh, W=lasagne.init.GlorotUniform())
#l_hidden_drop = lasagne.layers.DropoutLayer(l_hidden, p=0.5)
l_out = lasagne.layers.DenseLayer(l_hidden, num_units=1, nonlinearity=lasagne.nonlinearities.sigmoid, W=lasagne.init.GlorotUniform())

prediction = lasagne.layers.get_output(l_out).flatten()
#loss = lasagne.objectives.binary_crossentropy(prediction, target_var)
loss = lasagne.objectives.squared_error(prediction, target_var*0.99)

loss = loss.mean()

In [6]:
params = lasagne.layers.get_all_params(l_out, trainable=True)
print(params)
#updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)
updates = lasagne.updates.adam(loss, params)

[W, b, W, b]


In [7]:
test_prediction = lasagne.layers.get_output(l_out, deterministic=True).flatten()
test_loss = lasagne.objectives.binary_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()
test_acc = lasagne.objectives.binary_accuracy(T.gt(test_prediction, 0.5), target_var).mean()

In [8]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

prediction_fn = theano.function([input_var], T.gt(test_prediction, 0.))

In [9]:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [10]:
from random import randint
N = 100
for epoch in range(100000):
    idx = np.random.randint(2**9,size=N)
    input_data = all_input_data[idx, :]
    target_data = all_target_data[idx]
    if epoch%1 == 0:
        aloss, accuracy = val_fn(all_input_data, all_target_data)
        if epoch%100==0:
            print(epoch, "accuracy", accuracy, aloss)
        if accuracy == 1.0:
            break
    loss = train_fn(input_data, target_data)
print(epoch, "accuracy", val_fn(all_input_data, all_target_data)[1])

0 accuracy 0.5859375 0.6583781838417053
100 accuracy 0.708984375 0.5453886985778809
200 accuracy 0.828125 0.46027469635009766
300 accuracy 0.8359375 0.40126198530197144
400 accuracy 0.849609375 0.3687146306037903
500 accuracy 0.84765625 0.3505007028579712
600 accuracy 0.84375 0.33722493052482605
700 accuracy 0.859375 0.3303086757659912
800 accuracy 0.853515625 0.32213646173477173
900 accuracy 0.84765625 0.3188232183456421
1000 accuracy 0.861328125 0.3121160864830017
1100 accuracy 0.853515625 0.3075151741504669
1200 accuracy 0.857421875 0.30120742321014404
1300 accuracy 0.87109375 0.2947361171245575
1400 accuracy 0.8828125 0.2909179925918579
1500 accuracy 0.880859375 0.2801564931869507
1600 accuracy 0.884765625 0.2742465138435364
1700 accuracy 0.88671875 0.26584911346435547
1800 accuracy 0.892578125 0.25704246759414673
1900 accuracy 0.90234375 0.25057801604270935
2000 accuracy 0.908203125 0.24204590916633606
2100 accuracy 0.916015625 0.2357223927974701
2200 accuracy 0.91796875 0.2258404

In [11]:
for x in params:
    print(x.get_value())

[[  4.18483526e-01   2.69759119e-01  -7.59260170e-03   5.12706757e-01
    4.55930084e-01   2.79328287e-01   8.04584146e-01   3.88474204e-02
    4.17414069e-01  -5.56433916e-01   4.44707304e-01   6.61927164e-01
   -3.49529803e-01   6.29592240e-01  -2.40869656e-01   1.90079331e-01
    1.18805397e+00  -2.00313553e-01  -3.39206010e-01   4.57586423e-02
    8.72410119e-01  -6.92570031e-01   5.20162821e-01  -1.19220361e-01
   -2.57124305e-01   3.60738337e-01  -2.52967954e-01   2.16879901e-02
    2.45237380e-01  -3.28956217e-01  -2.83724010e-01  -2.21403107e-01
   -6.73699141e-01  -1.87377378e-01  -9.52876881e-02   2.64916360e-01
   -2.08849087e-01  -1.00696731e+00  -6.75794065e-01  -4.93810743e-01
    7.17632532e-01  -6.64196849e-01  -1.80892199e-01   9.95973349e-02
   -1.09519444e-01   6.33552790e-01  -7.73389399e-01  -3.06025475e-01
    6.89434171e-01  -7.62161016e-01  -4.28372651e-01   1.30543355e-02
   -8.49999189e-02  -3.06482241e-02  -3.22102845e-01   6.07555211e-01
   -1.57011449e-01  

In [12]:
params[0].get_value()[:, 6]

array([ 0.80458415,  0.39832583, -0.2455962 , -0.67453992,  0.14639822,
        1.73385942,  0.89292306,  0.36023572, -0.2375681 ], dtype=float32)